In [1]:
import sqlite3
import pandas as pd
import requests as rq
import os

In [2]:
print(sqlite3.version)
print(pd.__version__)

2.6.0
1.1.5


In [3]:
con = sqlite3.connect(":memory:")
cur = con.cursor()

https://www.cs.toronto.edu/~delve/data/boston/bostonDetail.html

In [4]:
link = 'https://raw.githubusercontent.com/eric-bunch/boston_housing/master/boston.csv'
file_path = os.getcwd() + '\\data\\boston.csv'

In [5]:
if not os.path.exists(os.getcwd() + '\\data\\boston.csv'):
    print('Файл отсутствует, скачиваем...')
    with open(file_path, 'wb') as f:
        f.write((rq.get(link)).content)
        f.close

In [6]:
data = pd.read_csv(file_path)
data.rename(columns = {'MDEV':'MEDV'}, inplace=True)

In [7]:
data.to_sql("boston", con, index=False, if_exists="replace")

In [8]:
pd.read_sql(
    """
    SELECT * FROM boston
    limit 5
    """,
    con,
)

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2


1. Выведите количество пустых значений по каждой из колонок (название колонки, кол-во пустых значений)

In [10]:
pd.read_sql(
    """
    SELECT
          count(CRIM   ) filter (where CRIM    is null) over() as CRIM_NULLS
        , count(ZN     ) filter (where ZN      is null) over() as ZN_NULLS
        , count(INDUS  ) filter (where INDUS   is null) over() as INDUS_NULLS
        , count(CHAS   ) filter (where CHAS    is null) over() as CHAS_NULLS
        , count(NOX    ) filter (where NOX     is null) over() as NOX_NULLS
        , count(RM     ) filter (where RM      is null) over() as RM_NULLS
        , count(AGE    ) filter (where AGE     is null) over() as AGE_NULLS
        , count(DIS    ) filter (where DIS     is null) over() as DIS_NULLS
        , count(RAD    ) filter (where RAD     is null) over() as RAD_NULLS
        , count(TAX    ) filter (where TAX     is null) over() as TAX_NULLS
        , count(PTRATIO) filter (where PTRATIO is null) over() as PTRATIO_NULLS
        , count(B      ) filter (where B       is null) over() as B_NULLS
        , count(LSTAT  ) filter (where LSTAT   is null) over() as LSTAT_NULLS
        , count(MEDV   ) filter (where MEDV    is null) over() as MEDV_NULLS
    FROM BOSTON
    limit 1
    """,
    con
)

,CRIM_NULLS,ZN_NULLS,INDUS_NULLS,CHAS_NULLS,NOX_NULLS,RM_NULLS,AGE_NULLS,DIS_NULLS,RAD_NULLS,TAX_NULLS,PTRATIO_NULLS,B_NULLS,LSTAT_NULLS,MEDV_NULLS
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


2. Выведите количество уникальных значений по каждой из колонок (название колонки, кол-во уникальных значений)

In [11]:
pd.read_sql(
    """
    SELECT 'CRIM' as "col_name" , (count(distinct CRIM)) as count_uniq_CRIM    FROM boston
    union
    SELECT 'ZN' , (count(distinct ZN)) as count_uniq_ZN      FROM boston
    union
    SELECT 'INDUS' , (count(distinct INDUS)) as count_uniq_INDUS   FROM boston
    union
    SELECT 'CHAS' , (count(distinct CHAS)) as count_uniq_CHAS    FROM boston
    union
    SELECT 'NOX' , (count(distinct NOX)) as count_uniq_NOX     FROM boston
    union
    SELECT 'RM' , (count(distinct RM)) as count_uniq_RM      FROM boston
    union
    SELECT 'AGE' , (count(distinct AGE)) as count_uniq_AGE     FROM boston
    union
    SELECT 'DIS' , (count(distinct DIS)) as count_uniq_DIS     FROM boston
    union
    SELECT 'RAD' , (count(distinct RAD)) as count_uniq_RAD     FROM boston
    union
    SELECT 'TAX' , (count(distinct TAX)) as count_uniq_TAX     FROM boston
    union
    SELECT 'PTRATIO' , (count(distinct PTRATIO)) as count_uniq_PTRATIO FROM boston
    union
    SELECT 'B' , (count(distinct B)) as count_uniq_B       FROM boston
    union
    SELECT 'LSTAT' , (count(distinct LSTAT)) as count_uniq_LSTAT   FROM boston
    union
    SELECT 'MEDV' , (count(distinct MEDV)) as count_uniq_MEDV    FROM boston
    """,
    con,
)

,col_name,count_uniq_CRIM
0,AGE,356
1,B,357
2,CHAS,2
3,CRIM,504
4,DIS,412
5,INDUS,76
6,LSTAT,455
7,MEDV,229
8,NOX,81
9,PTRATIO,46


3. Выведите колонки, у которых медиана равна минимальному значению (название колонки).
Напишите какой вывод можно сделать по данным в этих колонках

In [12]:
pd.read_sql(
    """
    with crim_group as (SELECT
                              CRIM   , ntile(2) over (order by CRIM    asc) as CRIM_gr
                            , ZN     , ntile(2) over (order by ZN      asc) as ZN_gr
                            , INDUS  , ntile(2) over (order by INDUS   asc) as INDUS_gr
                            , CHAS   , ntile(2) over (order by CHAS    asc) as CHAS_gr
                            , NOX    , ntile(2) over (order by NOX     asc) as NOX_gr
                            , RM     , ntile(2) over (order by RM      asc) as RM_gr
                            , AGE    , ntile(2) over (order by AGE     asc) as AGE_gr
                            , DIS    , ntile(2) over (order by DIS     asc) as DIS_gr
                            , RAD    , ntile(2) over (order by RAD     asc) as RAD_gr
                            , TAX    , ntile(2) over (order by TAX     asc) as TAX_gr
                            , PTRATIO, ntile(2) over (order by PTRATIO asc) as PTRATIO_gr
                            , B      , ntile(2) over (order by B       asc) as B_gr
                            , LSTAT  , ntile(2) over (order by LSTAT   asc) as LSTAT_gr
                            , MEDV   , ntile(2) over (order by MEDV    asc) as MEDV_gr
                        FROM boston)
    select
        trim (case when (min(CRIM   ) over(order by CRIM    asc)) = (((max(CRIM   ) filter(where CRIM_gr    = 1) over ()) + (min(CRIM   ) filter(where CRIM_gr    = 2) over ()))/2.0) then 'CRIM ' else '' end
        || case when (min(ZN     ) over(order by ZN      asc)) = (((max(ZN     ) filter(where ZN_gr      = 1) over ()) + (min(ZN     ) filter(where ZN_gr      = 2) over ()))/2.0) then 'ZN ' else '' end
        || case when (min(INDUS  ) over(order by INDUS   asc)) = (((max(INDUS  ) filter(where INDUS_gr   = 1) over ()) + (min(INDUS  ) filter(where INDUS_gr   = 2) over ()))/2.0) then 'INDUS ' else '' end
        || case when (min(CHAS   ) over(order by CHAS    asc)) = (((max(CHAS   ) filter(where CHAS_gr    = 1) over ()) + (min(CHAS   ) filter(where CHAS_gr    = 2) over ()))/2.0) then 'CHAS ' else '' end
        || case when (min(NOX    ) over(order by NOX     asc)) = (((max(NOX    ) filter(where NOX_gr     = 1) over ()) + (min(NOX    ) filter(where NOX_gr     = 2) over ()))/2.0) then 'NOX ' else '' end
        || case when (min(RM     ) over(order by RM      asc)) = (((max(RM     ) filter(where RM_gr      = 1) over ()) + (min(RM     ) filter(where RM_gr      = 2) over ()))/2.0) then 'RM ' else '' end
        || case when (min(AGE    ) over(order by AGE     asc)) = (((max(AGE    ) filter(where AGE_gr     = 1) over ()) + (min(AGE    ) filter(where AGE_gr     = 2) over ()))/2.0) then 'AGE ' else '' end
        || case when (min(DIS    ) over(order by DIS     asc)) = (((max(DIS    ) filter(where DIS_gr     = 1) over ()) + (min(DIS    ) filter(where DIS_gr     = 2) over ()))/2.0) then 'DIS ' else '' end
        || case when (min(RAD    ) over(order by RAD     asc)) = (((max(RAD    ) filter(where RAD_gr     = 1) over ()) + (min(RAD    ) filter(where RAD_gr     = 2) over ()))/2.0) then 'RAD ' else '' end
        || case when (min(TAX    ) over(order by TAX     asc)) = (((max(TAX    ) filter(where TAX_gr     = 1) over ()) + (min(TAX    ) filter(where TAX_gr     = 2) over ()))/2.0) then 'TAX ' else '' end
        || case when (min(PTRATIO) over(order by PTRATIO asc)) = (((max(PTRATIO) filter(where PTRATIO_gr = 1) over ()) + (min(PTRATIO) filter(where PTRATIO_gr = 2) over ()))/2.0) then 'PTRATIO ' else '' end
        || case when (min(B      ) over(order by B       asc)) = (((max(B      ) filter(where B_gr       = 1) over ()) + (min(B      ) filter(where B_gr       = 2) over ()))/2.0) then 'B ' else '' end
        || case when (min(LSTAT  ) over(order by LSTAT   asc)) = (((max(LSTAT  ) filter(where LSTAT_gr   = 1) over ()) + (min(LSTAT  ) filter(where LSTAT_gr   = 2) over ()))/2.0) then 'LSTAT ' else '' end
        || case when (min(MEDV   ) over(order by MEDV    asc)) = (((max(MEDV   ) filter(where MEDV_gr    = 1) over ()) + (min(MEDV   ) filter(where MEDV_gr    = 2) over ()))/2.0) then 'MEDV ' else '' end) as 'cols'
    from
        crim_group
    limit 1
    """,
    con,
)

,cols
0,ZN CHAS


4. Выведите разницу между среднем количеством комнат(RM) в домах с самой дорогой стоимостью(MEDV) и 25 самыми дешевыми домами. 
Аналогично по 50, 100, 200, 300 самыми дешевыми домами. (кол-во домов(25,50,100,200,300), среднее кол-во комнат в них, среднее кол-во комнат в самых дорогих, разница).
Напишите влияет ли кол-во комнат на стоимость и как сильно.

In [13]:
pd.read_sql(
    """
    with avg_rm_min_25_medv as (select avg(rm) min_avg_rm from (SELECT rm, row_number() over (order by medv asc) as rn FROM boston limit 25))
    , boston_max_medv as (select max(medv) over () as max_medv from boston limit 1)
    select
        avg(rm) - min_avg_rm as delta
    from boston, avg_rm_min_25_medv, boston_max_medv
    where medv = max_medv
    """,
    con,
)

,delta
0,1.73616


5. Выведите ранги значений колонки LSTAT(процент населения с более низким статусом) в домах с самой дорогой стоимостью (значение LSTAT, стоимость, ранг). Напишите какой вывод можно сделать по этим данным.

In [14]:
pd.read_sql(
    """
    select * from (SELECT
        medv
        , lstat
        , rank() over (order by lstat asc) as lstat_rank
    FROM boston)
    where 1=1
        and medv = (select max(medv) from boston)
    --limit 5
    """,
    con,
) 

,medv,lstat,lstat_rank
0,50.0,1.73,1
1,50.0,1.92,2
2,50.0,2.88,6
3,50.0,2.96,8
4,50.0,2.97,9
5,50.0,3.16,15
6,50.0,3.26,17
7,50.0,3.32,18
8,50.0,3.70,26
9,50.0,3.73,27


## Вывод.
Процент более низкого статуса населения среди самого дорого жилья в общей статистике не превышает 10%

6. Выведите среднюю стоимость домов граничащих с рекой(CHAS) и нет (граничит/не граничит, стоимость)

In [15]:
pd.read_sql(
    '''
    SELECT distinct
        chas
        , avg(medv) over(partition by chas) as avg_medv_by_chas
    FROM boston
    ''',
    con,
)

,CHAS,avg_medv_by_chas
0,0.0,22.093843
1,1.0,28.440000


7. Выведите все колонки, у которых среднее значение выше, когда дом граничит с рекой (название колонки). Напишите какой вывод можно сделать по этим данным.

In [16]:
pd.read_sql(
    """
        SELECT group_concat(col_name, ', ') as cols from (
            select 'CRIM' as col_name, (avg(CRIM) filter (where chas = 1.0) over()) - (avg(CRIM) filter (where chas = 0.0) over()) as avg_by_chas from boston union
            select 'ZN' as col_name, (avg(ZN) filter (where chas = 1.0) over()) - (avg(ZN) filter (where chas = 0.0) over()) as avg_by_chas from boston union
            select 'INDUS' as col_name, (avg(INDUS) filter (where chas = 1.0) over()) - (avg(INDUS) filter (where chas = 0.0) over()) as avg_by_chas from boston union
            select 'NOX' as col_name, (avg(NOX) filter (where chas = 1.0) over()) - (avg(NOX) filter (where chas = 0.0) over()) as avg_by_chas from boston union
            select 'RM' as col_name, (avg(RM) filter (where chas = 1.0) over()) - (avg(RM) filter (where chas = 0.0) over()) as avg_by_chas from boston union
            select 'AGE' as col_name, (avg(AGE) filter (where chas = 1.0) over()) - (avg(AGE) filter (where chas = 0.0) over()) as avg_by_chas from boston union
            select 'DIS' as col_name, (avg(DIS) filter (where chas = 1.0) over()) - (avg(DIS) filter (where chas = 0.0) over()) as avg_by_chas from boston union
            select 'RAD' as col_name, (avg(RAD) filter (where chas = 1.0) over()) - (avg(RAD) filter (where chas = 0.0) over()) as avg_by_chas from boston union
            select 'TAX' as col_name, (avg(TAX) filter (where chas = 1.0) over()) - (avg(TAX) filter (where chas = 0.0) over()) as avg_by_chas from boston union
            select 'PTRATIO' as col_name, (avg(PTRATIO) filter (where chas = 1.0) over()) - (avg(PTRATIO) filter (where chas = 0.0) over()) as avg_by_chas from boston union
            select 'B' as col_name, (avg(B) filter (where chas = 1.0) over()) - (avg(B) filter (where chas = 0.0) over()) as avg_by_chas from boston union
            select 'LSTAT' as col_name, (avg(LSTAT) filter (where chas = 1.0) over()) - (avg(LSTAT) filter (where chas = 0.0) over()) as avg_by_chas from boston union
            select 'MEDV' as col_name, (avg(MEDV) filter (where chas = 1.0) over()) - (avg(MEDV) filter (where chas = 0.0) over()) as avg_by_chas from boston
            ) boston_avg
        where
            avg_by_chas > 0
    """,
    con,
)

,cols
0,"AGE, B, INDUS, MEDV, NOX, RM"


### Вывод:
Доля жилых домов, построенных до 1940 года больше у тех, которые находятся возле реки, что вполне логично, так как застройка городов на ранних этапах производится по мере удаления от реки.
При этом доля афроамериканцев, проживающих в домах возле реки больше.
А также доля земли под нежилой сектор ближе к реке, а соответственно к центру города.
Из-за этого и средняя стоимость жилья выше, так как центр. Из-за центра и выбросы углекислого газа выше.
Выше так же отмечается среднее количество комнат в квартире.

8. Выведите значения долей промышленной застройки(INDUS), концентрации оксидов азота(NOX) и по их перцентилям - 10, 20 ... 100 ( перцетиль(10,20...100),значение INDUS, значение NOX). Напишите прослеживается ли между ними взаимосвязь

In [17]:
pd.read_sql(
    """
    select a.*, b.indus_percentile from
        (
            with bost_ntile as (
                        SELECT
                            nox, ntile(10) over(order by nox)  as part_nox
                        FROM
                            boston
                    )
            select
                part_nox*10 as percentile
                , sum(sum(nox)) over(rows between unbounded preceding and current row) as nox_percentile
            from
                bost_ntile
            group by part_nox
        ) a join  (
             with bost_ntile as (
                        SELECT
                            indus, ntile(10) over(order by indus)  as part_indus
                        FROM
                            boston
                    )
            select
                part_indus*10 as percentile
                , sum(sum(indus)) over(rows between unbounded preceding and current row) as indus_percentile
            from
                bost_ntile
            group by part_indus
        ) b on b.percentile = a.percentile
    """,
    con,
)

,percentile,nox_percentile,indus_percentile
0,10,20.7923,102.82
1,20,42.9341,288.73
2,30,66.0597,556.29
3,40,91.2107,886.35
4,50,117.7367,1307.41
5,60,145.7087,1855.62
6,70,175.2807,2717.91
7,80,206.9577,3622.91
8,90,241.6607,4538.27
9,100,280.6757,5635.21


## Вывод
С ростом доли акров неторгового бизнеса растут и выбросы углекислого газа в атмосферу, но рост при этом разный, нет прямой пропорциональной зависимости.